<a href="https://colab.research.google.com/github/aSafarpoor/KGCLG/blob/main/0_Read_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import random
import numpy as np
from tqdm import tqdm 
import pandas as pd
import json
from collections import Counter
import pandas as pd
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/My Drive/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive


In [4]:
# !mkdir KGCLG
%cd KGCLG

/content/drive/My Drive/KGCLG


## read data

In [11]:
!wget https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/Software.json.gz --no-check-certificate

--2022-10-31 15:19:06--  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/Software.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 98221392 (94M) [application/x-gzip]
Saving to: ‘Software.json.gz’

Software.json.gz    100%[===================>]  93.67M  60.8MB/s    in 1.5s    

2022-10-31 15:19:08 (60.8 MB/s) - ‘Software.json.gz’ saved [98221392/98221392]



In [12]:
!wget https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Software.json.gz --no-check-certificate

--2022-10-31 15:19:11--  https://jmcauley.ucsd.edu/data/amazon_v2/metaFiles2/meta_Software.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 15240894 (15M) [application/x-gzip]
Saving to: ‘meta_Software.json.gz’

meta_Software.json. 100%[===================>]  14.53M  56.0MB/s    in 0.3s    

2022-10-31 15:19:12 (56.0 MB/s) - ‘meta_Software.json.gz’ saved [15240894/15240894]



In [57]:
!gunzip -k meta_Software.json.gz
!gunzip -k Software.json.gz
!rm meta_Software.json.gz
!rm Software.json.gz

gzip: meta_Software.json.gz: No such file or directory
gzip: Software.json.gz: No such file or directory
rm: cannot remove 'meta_Software.json.gz': No such file or directory
rm: cannot remove 'Software.json.gz': No such file or directory


In [58]:
!ls

meta_Software.json  Software.json


In [78]:
meta = pd.read_json('meta_Software.json', lines=True)
ui = pd.read_json('Software.json', lines=True)

In [79]:
ui.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image'],
      dtype='object')

In [80]:
print(len(ui))
ui = ui[ui.overall>=4]
ui = ui[['reviewerID','asin']]
print(len(ui))


459436
286048


In [81]:
ui = ui.dropna()
ui = ui.drop_duplicates()
print(len(ui))

280490


In [82]:
ui = ui.reset_index()
del ui['index']

In [83]:
ui.head(5)

,reviewerID,asin
0,A240ORQ2LF9LUI,0077613252
1,A1YCCU0YRLS0FE,0077613252
2,A2JZTTBSLS1QXV,0077775473
3,AGN3T5ERYJN5A,0132147556
4,A35UC8LHA6TILH,0132147556


In [84]:
meta.head(3)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,[],,[],,HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK ...,[],,HOLT. RINEHART AND WINSTON,[],"25,550 in Software (",[],Software,,</div>,.a-box-inner{background-color:#fff}#alohaBuyBo...,0030672120,[],[],NaN
1,[],,"[, <b>Latin rhythms that will get your kids si...",,"Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",[],,McGraw Hill,[],"15,792 in Software (",[],Software,,</div>,,0071480935,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,[],,[<b>Connect is the only integrated learning sy...,,Connect with LearnSmart Access Card for Microb...,[],,McGraw-Hill Science/Engineering/Math,[],"16,900 in Software (",[],Software,,</div>,,007329506X,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [85]:
meta.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')

In [86]:
meta = meta[['asin','description','title','brand',"imageURL","imageURLHighRes"]]

In [87]:
meta = meta.dropna()

In [88]:
meta = meta.reset_index()
del meta['index']

In [89]:
meta.head(5)

,asin,description,title,brand,imageURL,imageURLHighRes
0,0030672120,[],HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK ...,HOLT. RINEHART AND WINSTON,[],[]
1,0071480935,"[, <b>Latin rhythms that will get your kids si...","Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",McGraw Hill,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,007329506X,[<b>Connect is the only integrated learning sy...,Connect with LearnSmart Access Card for Microb...,McGraw-Hill Science/Engineering/Math,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,0073513458,[],LearnSmart Standalone Access Card for Prescott...,McGraw-Hill Education,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,0073525758,[<i>Anatomy &amp; Physiology Revealed Cat</i> ...,Anatomy &amp; Physiology Revealed Student Acce...,McGraw-Hill Education,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [90]:
print(len(meta))
meta = meta[meta['description'].map(lambda d: len(d)) > 0]
meta = meta[meta['title'].map(lambda d: len(d)) > 0]
meta = meta[meta['asin'].map(lambda d: len(d)) > 0]
meta = meta[meta['brand'].map(lambda d: len(d)) > 0]
meta = meta[meta['imageURL'].map(lambda d: len(d)) > 0]
meta = meta[meta['imageURLHighRes'].map(lambda d: len(d)) > 0]
print(len(meta))
meta.head(5)

26790
10279


,asin,description,title,brand,imageURL,imageURLHighRes
1,0071480935,"[, <b>Latin rhythms that will get your kids si...","Sing, Watch, &amp; Learn Spanish (DVD + Guide)...",McGraw Hill,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,007329506X,[<b>Connect is the only integrated learning sy...,Connect with LearnSmart Access Card for Microb...,McGraw-Hill Science/Engineering/Math,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,0073525758,[<i>Anatomy &amp; Physiology Revealed Cat</i> ...,Anatomy &amp; Physiology Revealed Student Acce...,McGraw-Hill Education,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
5,0077340701,"[John Coburn grew up in the Hawaiian Islands, ...",Connect Math by ALEKS Access Card 52 Weeks for...,McGraw-Hill Science/Engineering/Math,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
6,0077369823,[<b>Connect is the only integrated learning sy...,Connect Access Card for Microbiology Fundament...,Brand: McGraw-Hill Science/Engineering/Math,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


## meta clearance

In [91]:
len(meta),len(ui)

(10279, 280490)

In [92]:
def description_improvement(l):
    s = ' '.join(l)
    return(s)
meta['description'] = meta['description'].apply(description_improvement) 

In [93]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

class Normalizer:
    # based on https://www.geeksforgeeks.org/normalizing-textual-data-with-python/
    def __init__(self):
        # import regex
        self.stop_words = set(stopwords.words('english'))

    def text_normalizer(self,text):
        if type(text) == type([]):
            text = text[0]
        text = str(text)
        # convert to lower case
        lower_string = text.lower()
        # remove numbers
        no_number_string = re.sub(r'\d+','',lower_string)
        # remove all punctuation except words and space
        no_punc_string = re.sub(r'[^\w\s]','', no_number_string)
        # remove white spaces
        no_wspace_string = no_punc_string.strip()
        # convert string to list of words
        lst_string = [no_wspace_string][0].split()
        # remove stopwords
        no_stpwords_string=""
        for i in lst_string:
            if not i in self.stop_words:
                no_stpwords_string += i+' '
        # removing last space
        no_stpwords_string = no_stpwords_string[:-1]
        output = no_stpwords_string
        return output

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [94]:
normalizer = Normalizer()

In [95]:
meta["description"] = meta["description"].apply(normalizer.text_normalizer)
meta["title"] = meta["title"].apply(normalizer.text_normalizer)

In [97]:
def brand_normalizer(s):
    s = s.replace("Brand:","")
    s = s.replace(" ","")
    s = s.split("/")
    return s

In [98]:
meta["brand"] = meta["brand"].apply(brand_normalizer)

In [101]:
def url_normalizer(s):
    return s[0]

In [103]:
meta["imageURL"] = meta["imageURL"].apply(url_normalizer)
meta["imageURLHighRes"] = meta["imageURLHighRes"].apply(url_normalizer)

In [105]:
meta['text'] = meta[['title', 'description','title']].apply(lambda x: ''.join(x), axis=1)

In [106]:
meta.head(5)

,asin,description,title,brand,imageURL,imageURLHighRes,text
1,0071480935,blatin rhythms get kids singing spanishb ising...,sing watch amp learn spanish dvd guide fun mus...,[McGrawHill],https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,sing watch amp learn spanish dvd guide fun mus...
2,007329506X,bconnect integrated learning system empowers s...,connect learnsmart access card microbiology fu...,"[McGraw-HillScience, Engineering, Math]",https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,connect learnsmart access card microbiology fu...
4,0073525758,ianatomy amp physiology revealed cati ultimate...,anatomy amp physiology revealed student access...,[McGraw-HillEducation],https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,anatomy amp physiology revealed student access...
5,0077340701,john coburn grew hawaiian islands seventh sixt...,connect math aleks access card weeks college a...,"[McGraw-HillScience, Engineering, Math]",https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,connect math aleks access card weeks college a...
6,0077369823,bconnect integrated learning system empowers s...,connect access card microbiology fundamentals ...,"[McGraw-HillScience, Engineering, Math]",https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,connect access card microbiology fundamentals ...


In [108]:
meta = meta.reset_index()
del meta['index']

In [111]:
for i in meta.iloc[1]:
    print(i)

007329506X
bconnect integrated learning system empowers students continuously adapting deliver precisely need need need class time engaging effectivebbr kelly cowan celebrated th anniversary miami university middletown open admissions campus ohio received phd university louisville later worked university maryland university groningen netherlands specializes teaching microbiology nonmajors especially prenursing allied health students fell love microbiology pursuing undergraduate degree dental hygiene made personal mission hear nurses dental hygienists encounters everyday situations exclaim loved microbiology class
connect learnsmart access card microbiology fundamentals clinical approach
['McGraw-HillScience', 'Engineering', 'Math']
https://images-na.ssl-images-amazon.com/images/I/51z1MKLSD3L._SX38_SY50_CR,0,0,38,50_.jpg
https://images-na.ssl-images-amazon.com/images/I/51z1MKLSD3L.jpg
connect learnsmart access card microbiology fundamentals clinical approachbconnect integrated learning 